# **Анализ динамики котировок Роснефти (Шаг №1 Парсинг данных)**

# **Импортирование библиотек**

In [4]:
%%capture
!pip install apimoex
!pip install requests pandas
!pip install requests beautifulsoup4
!pip install mpl_finance

In [5]:
import pandas as pd
import numpy as np
import requests
import apimoex
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
import re
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import scipy.stats as sts
from mpl_finance import candlestick_ohlc 
import matplotlib.dates as mpl_dates 

# **Парсинг данных с MOEX API**

Данные были спарсены с сайта https://www.moex.com/ при помощи их MOEX API, который сначала необходимо было импортировать, а потом написать небольшой парсер, который был написан под впечатлением от
* https://habr.com/ru/articles/759922/
* https://pypi.org/project/apimoex/
* https://github.com/mbk-dev/moex

После чего был написан парсер. Работу которого частично закомментировал

In [6]:
def fetch_moex_data(security, start_date, end_date):
    # Формирование URL для доступа к данным в формате JSON для указанной ценной бумаги на MOEX вытащенные с HTML кода страницы
    url = f'https://iss.moex.com/iss/history/engines/stock/markets/shares/securities/{security}.json'
    # Инициализация пустого списка для хранения DataFrame, созданных из полученных данных.
    all_data = []
    # Инициализация начальной точки для пагинации данных.
    start = 0
    # Начало бесконечного цикла для получения всех данных в несколько запросов.

    while True:
        params = {'from': start_date, 'till': end_date, 'start': start}
        # Создание сессии для управления соединениями и повторами попыток.
        with requests.Session() as session:
            session.mount('https://', HTTPAdapter(max_retries=Retry(total=3, backoff_factor=1)))
            response = session.get(url, params=params)
            # Генерация исключения, если запрос был неудачен.
            response.raise_for_status()
            data = response.json()
            # Извлечение части исторических данных из ответа.
            history_data = data['history']['data']
            # Если данных нет, прерывание цикла (конец данных).
            if not history_data:
                break
            # Извлечение названий столбцов для данных.

            columns = data['history']['columns']
            # Создание DataFrame из исторических данных и названий столбцов, добавление в список.
            df = pd.DataFrame(history_data, columns=columns)
            all_data.append(df)
            # Увеличение параметра start на количество возвращенных элементов для правильной пагинации.
            start += len(history_data)

    # Объединение всех собранных DataFrame в один.
    full_df = pd.concat(all_data, ignore_index=True)
    # Преобразование указанных столбцов в числовые типы, обработка нечисловых данных как пропущенных значений.

    for col in ['NUMTRADES', 'VOLUME', 'OPEN', 'LOW', 'HIGH', 'LEGALCLOSEPRICE', 'WAPRICE', 'CLOSE']:
        full_df[col] = pd.to_numeric(full_df[col], errors='coerce')

    return full_df

In [7]:
security = 'ROSN'
start_date = '2021-01-01'
end_date = '2024-04-01'
df_ROSN = fetch_moex_data(security, start_date, end_date)
df_ROSN.to_csv(f"{security}_trades_{start_date}_to_{end_date}.csv", index=False)

Аналогично были вытащены данные по SBER

In [8]:
security = 'SBER'
start_date = '2021-01-01'
end_date = '2024-04-01'
df_SBER = fetch_moex_data(security, start_date, end_date)
df_SBER.to_csv(f"{security}_trades_{start_date}_to_{end_date}.csv", index=False)

Используя ранее написанный парсер мы также решили добавить данные за аналогичный период котировок еще одной нефтяной компании "Лукойл". В данном разделе идет сугубо парсинг, логика и обоснование выбранных данных будут представлены в следующем разделе.

In [9]:
security = 'LKOH'
start_date = '2021-01-01'
end_date = '2024-04-01'
df_LKOH = fetch_moex_data(security, start_date, end_date)
df_LKOH.to_csv(f"{security}_trades_{start_date}_to_{end_date}.csv", index=False)

In [10]:
df_ROSN.head()

,BOARDID,TRADEDATE,SHORTNAME,SECID,NUMTRADES,VALUE,OPEN,LOW,HIGH,LEGALCLOSEPRICE,...,MARKETPRICE2,MARKETPRICE3,ADMITTEDQUOTE,MP2VALTRD,MARKETPRICE3TRADESVALUE,ADMITTEDVALUE,WAVAL,TRADINGSESSION,CURRENCYID,TRENDCLSPR
0,SMAL,2021-01-04,Роснефть,ROSN,6,7.882800e+03,435.75,434.40,440.0,NaN,...,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.0,0,3,SUR,0.57
1,TQBR,2021-01-04,Роснефть,ROSN,22434,2.803388e+09,435.85,433.00,442.6,442.05,...,437.65,437.65,442.05,2.476802e+09,2.476802e+09,2476802287.0,0,3,SUR,0.21
2,SMAL,2021-01-05,Роснефть,ROSN,5,5.299750e+03,439.00,437.85,452.5,NaN,...,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.0,0,3,SUR,2.84
3,TQBR,2021-01-05,Роснефть,ROSN,23452,2.974642e+09,436.00,433.30,450.7,445.60,...,440.55,440.55,445.6,2.338930e+09,2.338930e+09,2338929568.5,0,3,SUR,3.06
4,SMAL,2021-01-06,Роснефть,ROSN,8,7.156250e+03,446.00,442.25,452.8,NaN,...,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.0,0,3,SUR,0.07


In [11]:
df_SBER.head()

,BOARDID,TRADEDATE,SHORTNAME,SECID,NUMTRADES,VALUE,OPEN,LOW,HIGH,LEGALCLOSEPRICE,...,MARKETPRICE2,MARKETPRICE3,ADMITTEDQUOTE,MP2VALTRD,MARKETPRICE3TRADESVALUE,ADMITTEDVALUE,WAVAL,TRADINGSESSION,CURRENCYID,TRENDCLSPR
0,SMAL,2021-01-04,Сбербанк,SBER,17,1.538091e+04,274.78,271.18,279.15,NaN,...,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.0,0,3,SUR,0.64
1,TQBR,2021-01-04,Сбербанк,SBER,74192,1.343454e+10,274.67,270.61,276.60,276.00,...,275.06,275.06,276.0,1.099544e+10,1.099544e+10,10995442585.299999,0,3,SUR,0.13
2,SMAL,2021-01-05,Сбербанк,SBER,6,2.192670e+03,273.30,273.00,277.60,NaN,...,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.0,0,3,SUR,1.77
3,TQBR,2021-01-05,Сбербанк,SBER,74787,1.244994e+10,272.69,270.28,275.70,273.82,...,272.97,272.97,273.82,1.133575e+10,1.133575e+10,11335745413.799999,0,3,SUR,1.18
4,SMAL,2021-01-06,Сбербанк,SBER,8,6.356530e+03,277.36,273.00,277.36,NaN,...,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.0,0,3,SUR,-1.30


In [12]:
df_LKOH.head()

,BOARDID,TRADEDATE,SHORTNAME,SECID,NUMTRADES,VALUE,OPEN,LOW,HIGH,LEGALCLOSEPRICE,...,MARKETPRICE2,MARKETPRICE3,ADMITTEDQUOTE,MP2VALTRD,MARKETPRICE3TRADESVALUE,ADMITTEDVALUE,WAVAL,TRADINGSESSION,CURRENCYID,TRENDCLSPR
0,TQBR,2021-01-04,ЛУКОЙЛ,LKOH,25148,4.077814e+09,5238.5,5186.5,5274.5,5243.5,...,5243.5,5243.5,5243.5,3.534440e+09,3.534440e+09,3534439785.5,0,3,SUR,0.83
1,TQBR,2021-01-05,ЛУКОЙЛ,LKOH,33260,6.222501e+09,5214.5,5180.0,5359.0,5311.0,...,5265.0,5265.0,5311.0,5.159927e+09,5.159927e+09,5159927106.5,0,3,SUR,2.80
2,TQBR,2021-01-06,ЛУКОЙЛ,LKOH,38447,7.490176e+09,5375.0,5304.0,5401.0,5395.0,...,5367.5,5367.5,5395.0,6.804506e+09,6.804506e+09,6804506193.5,0,3,SUR,0.34
3,TQBR,2021-01-08,ЛУКОЙЛ,LKOH,55384,1.140255e+10,5450.0,5425.0,5557.5,5521.0,...,5506.5,5506.5,5521.0,1.071762e+10,1.071762e+10,10717623852.0,0,3,SUR,3.35
4,TQBR,2021-01-11,ЛУКОЙЛ,LKOH,49387,1.140988e+10,5510.0,5451.0,5639.0,5600.0,...,5571.0,5571.0,5600.0,1.100347e+10,1.100347e+10,11003470110.0,0,3,SUR,0.76


In [13]:
print(df_ROSN.shape)
print(df_SBER.shape)
print(df_LKOH.shape)

(1005, 23)
(1641, 23)
(836, 23)


Мы видим разницу в размерности данных, эта проблема будет решена и описана в следующем разделе, в котором будет проведена финальная обработка данных.

После парсинга данных о компаниях было принято решено спарсить для дальнейшего анализа данные, касающиеся фьючерсов на нефть

In [14]:
url = "https://www.moex.com/ru/forts/contractbaseresults.aspx?day1=20210101&day2=20240401&base=BR"

response = requests.get(url)
response.raise_for_status()

soup = BeautifulSoup(response.text, 'html.parser')

table = soup.find('table', {'class': 'tablels'})

rows = table.find_all('tr')

data = []
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append(cols)

df_BR = pd.DataFrame(data)
df_BR.columns = ['Date', 'FuturesCode', 'WeightedAvgPrice', 'SettlementPrice', 'FirstDealPrice',
              'MaxPrice', 'MinPrice', 'LastDealPrice', 'Change', 'VolumeLastDeal',
              'NumOfDeals', 'TradingVolume RUB', 'TradingVolume contr', 'OpenPositionsVolume RUB', 'OpenPositionsVolume contr', 'MarginRequirements']
df_BR.to_csv(f"BR_trades_{start_date}_to_{end_date}.csv", index=False)

In [15]:
df_BR[2:]

,Date,FuturesCode,WeightedAvgPrice,SettlementPrice,FirstDealPrice,MaxPrice,MinPrice,LastDealPrice,Change,VolumeLastDeal,NumOfDeals,TradingVolume RUB,TradingVolume contr,OpenPositionsVolume RUB,OpenPositionsVolume contr,MarginRequirements
2,04.01.2021,BRF1,"51,28","51,26","51,35","51,53","51,14","51,27","-0,27 %",50,15 230,9 083 037 005,240 246,-,-,10 097
3,05.01.2021,BRG1,"51,73","52,87","51,24","53,06","50,58","52,86","+2,58 %",1,154 728,74 458 569 937,1 937 554,14 354 505 023,365 456,10 978
4,06.01.2021,BRG1,"53,76","54,33","52,94","54,62","52,67","54,38","+2,88 %",1,193 366,98 866 320 215,2 486 797,17 508 172 524,435 758,11 107
5,08.01.2021,BRG1,"54,69","55,42","54,31","55,49","53,11","55,47","+2,00 %",5,142 468,71 925 211 898,1 768 768,18 937 569 926,459 570,9 046
6,11.01.2021,BRG1,"55,41","55,56","55,43","56,16","55,01","55,57","+0,18 %",1,125 354,56 102 626 259,1 354 714,19 055 391 580,458 902,9 078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823,29.03.2024,BRJ4,"87,4","87,4","87,27","87,54","87,21","87,4","+0,14 %",1,12 170,4 200 670 252,52 042,5 551 054 394,68 772,16 714
824,01.04.2024,BRJ4,"87,42","87,42","87,4","87,47","87,39","87,42","+0,02 %",60,5 065,2 536 892 203,31 427,-,-,16 696
825,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
826,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


Мы также заполучили данные касающиеся фьючерсов на доллар, за рассматриваемый период (Si фьючерсный контракт на курс доллар США-Российский рубль)

In [16]:
url = 'https://www.moex.com/ru/forts/contractbaseresults.aspx?day1=20210101&day2=20240401&base=Si'

response = requests.get(url)
response.raise_for_status()

soup = BeautifulSoup(response.text, 'html.parser')

table = soup.find('table', {'class': 'tablels'})

rows = table.find_all('tr')

data = []
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append(cols)

df_SI = pd.DataFrame(data)
df_SI.columns = ['Date', 'FuturesCode', 'WeightedAvgPrice', 'SettlementPrice', 'FirstDealPrice',
              'MaxPrice', 'MinPrice', 'LastDealPrice', 'Change', 'VolumeLastDeal',
              'NumOfDeals', 'TradingVolume RUB', 'TradingVolume contr', 'OpenPositionsVolume RUB', 'OpenPositionsVolume contr', 'MarginRequirements']
df_SI.to_csv(f"SI_trades_{start_date}_to_{end_date}.csv", index=False)

In [17]:
df_SI

,Date,FuturesCode,WeightedAvgPrice,SettlementPrice,FirstDealPrice,MaxPrice,MinPrice,LastDealPrice,Change,VolumeLastDeal,NumOfDeals,TradingVolume RUB,TradingVolume contr,OpenPositionsVolume RUB,OpenPositionsVolume contr,MarginRequirements
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,04.01.2021,SiH1,74 151,74 468,75 175,75 175,73 555,74 473,"-0,97 %",1,628 063,234 450 156 953,3 161 789,195 876 605 928,2 630 346,6 165
3,05.01.2021,SiH1,75 122,74 747,74 500,75 818,74 403,74 792,"+0,43 %",1,911 840,304 227 626 196,4 049 800,183 061 382 760,2 449 080,6 187
4,06.01.2021,SiH1,74 323,74 506,74 780,74 780,73 908,74 498,"-0,39 %",1,768 465,235 490 576 934,3 168 486,184 370 163 408,2 474 568,6 165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823,29.03.2024,SiM4,94 355,94 463,94 822,94 899,94 050,94 461,"-0,38 %",1,224 650,103 732 359 915,1 099 329,548 219 421 168,5 803 536,14 687
824,01.04.2024,SiM4,94 555,94 760,94 480,94 838,94 255,94 750,"+0,31 %",30,147 944,58 564 777 979,619 373,563 313 896 880,5 944 638,14 689
825,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
826,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


Логика взятых показателей будет описана в следующем разделе.

Нам также понадобится далее информация о индексе мосбиржи Moex. Индекс МосБиржи (ранее «Индекс ММВБ») — ценовой, взвешенный по рыночной капитализации композитный фондовый индекс, включающий 50 наиболее ликвидных акций крупнейших и динамично развивающихся российских эмитентов, виды экономической деятельности которых относятся к основным секторам экономики, представленных на Московской бирже. Перечень эмитентов и их вес в индексе пересматривается раз в квартал. Тикер — MCX: IMOEX

In [18]:
def fetch_moex_data(engine, market, security, date_from, date_till):
    base_url = f"https://iss.moex.com/iss/history/engines/{engine}/markets/{market}/securities/{security}.json"
    params = {
        'from': date_from,
        'till': date_till,
        'start': 0
    }
    all_data = []
    while True:
        response = requests.get(base_url, params=params)
        if response.status_code != 200:
            print(f"Ошибка запроса, статус код: {response.status_code}")
            break
        data = response.json()
        rows = data['history']['data']
        if not rows:
            break
        all_data.extend(rows)
        params['start'] += len(rows)
    
    columns = data['history']['columns']
    return pd.DataFrame(all_data, columns=columns)


In [19]:
engine = 'stock'
market = 'index'
security = 'IMOEX'
start_date = '2021-01-01'
end_date = '2024-04-01'

df_IMOEX = fetch_moex_data(engine, market, security, start_date, end_date)

In [20]:
df_IMOEX.to_csv(f"IMOEX_trades_{start_date}_to_{end_date}.csv", index=False)

In [21]:
df_IMOEX

,BOARDID,SECID,TRADEDATE,SHORTNAME,NAME,CLOSE,OPEN,HIGH,LOW,VALUE,DURATION,YIELD,DECIMALS,CAPITALIZATION,CURRENCYID,DIVISOR,TRADINGSESSION,VOLUME
0,SNDX,IMOEX,2021-01-04,Индекс МосБиржи,Индекс МосБиржи,3350.51,3306.12,3350.81,3304.17,6.687503e+10,0,0,2,1.701802e+13,RUB,5.079231e+09,3,None
1,SNDX,IMOEX,2021-01-05,Индекс МосБиржи,Индекс МосБиржи,3359.15,3327.94,3365.32,3301.88,7.166972e+10,0,0,2,1.706189e+13,RUB,5.079231e+09,3,None
2,SNDX,IMOEX,2021-01-06,Индекс МосБиржи,Индекс МосБиржи,3371.03,3373.37,3380.63,3344.43,7.145809e+10,0,0,2,1.712225e+13,RUB,5.079231e+09,3,None
3,SNDX,IMOEX,2021-01-08,Индекс МосБиржи,Индекс МосБиржи,3454.82,3390.23,3474.66,3390.23,1.321056e+11,0,0,2,1.754780e+13,RUB,5.079231e+09,3,None
4,SNDX,IMOEX,2021-01-11,Индекс МосБиржи,Индекс МосБиржи,3482.48,3455.67,3516.90,3436.66,1.432925e+11,0,0,2,1.768832e+13,RUB,5.079231e+09,3,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,SNDX,IMOEX,2024-03-26,Индекс МосБиржи,Индекс МосБиржи,3285.54,3286.20,3291.00,3275.80,5.978329e+10,0,0,2,5.796714e+12,RUB,1.764313e+09,3,None
802,SNDX,IMOEX,2024-03-27,Индекс МосБиржи,Индекс МосБиржи,3304.09,3291.70,3304.09,3280.03,5.142267e+10,0,0,2,5.829443e+12,RUB,1.764313e+09,3,None
803,SNDX,IMOEX,2024-03-28,Индекс МосБиржи,Индекс МосБиржи,3312.77,3308.26,3313.59,3301.52,6.164930e+10,0,0,2,5.844767e+12,RUB,1.764313e+09,3,None
804,SNDX,IMOEX,2024-03-29,Индекс МосБиржи,Индекс МосБиржи,3332.53,3313.17,3332.53,3305.22,5.929462e+10,0,0,2,5.879632e+12,RUB,1.764313e+09,3,None
